# Setup

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
import seaborn as sns
sns.set()
from tqdm import tqdm
from IPython.display import Image

from src.space.utils import get_coord_matrix, get_affine_transform
from src.model.gmu_social import GmuSocial
from src.visualization.utils import plot_commuter_status_count, plot_num_friendships

# World

In [2]:
world = gpd.read_file("../data/raw/campus/world.shp").set_index("Id").set_crs("epsg:2283", allow_override=True)
world.centroid.iloc[0].y

6987693.086325529

In [3]:
world.crs

<Derived Projected CRS: EPSG:2283>
Name: NAD83 / Virginia North (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - Virginia - counties of Arlington; Augusta; Bath; Caroline; Clarke; Culpeper; Fairfax; Fauquier; Frederick; Greene; Highland; King George; Loudoun; Madison; Orange; Page; Prince William; Rappahannock; Rockingham; Shenandoah; Spotsylvania; Stafford; Warren; Westmoreland.
- bounds: (-80.06, 37.77, -76.51, 39.46)
Coordinate Operation:
- name: SPCS83 Virginia North zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
from shapely.ops import transform
import pyproj

projector = pyproj.Transformer.from_crs(pyproj.CRS("epsg:2283"), pyproj.CRS("epsg:4269"), always_xy=True).transform

new_world = gpd.GeoSeries(transform(projector, world.iloc[0].geometry))
new_world

0    POLYGON ((-77.31308 38.83462, -77.29352 38.834...
dtype: geometry

# GMU Buildings

In [5]:
gmu_buildings = gpd.read_file("../data/raw/campus/Mason_bld.shp").fillna(0.0).set_index("Id").rename(columns={"NAME": "name"})\
                   .set_crs("epsg:2283", allow_override=True)
gmu_buildings.head()

,name,function,geometry
Id,,,
1,GEORGE W. JOHNSON CENTER,1.0,"POLYGON ((11822517.112 6987684.108, 11822557.0..."
2,DAVID KING HALL,1.0,"POLYGON ((11822800.554 6987724.175, 11822753.8..."
3,SCIENCE AND TECH I,1.0,"POLYGON ((11823106.160 6987428.086, 11823006.5..."
4,SCIENCE AND TECH II,1.0,"POLYGON ((11823144.777 6987485.867, 11823154.0..."
5,ENTERPRISE HALL,1.0,"POLYGON ((11822946.564 6987144.449, 11822939.8..."


In [18]:
gmu_buildings["function"].value_counts()

0.0    22
2.0    22
1.0    17
Name: function, dtype: int64

In [6]:
gmu_buildings["centroid"] = [(x,y) for x,y in zip(gmu_buildings.centroid.x , gmu_buildings.centroid.y)]

In [7]:
gmu_buildings.head()

,name,function,geometry,centroid
Id,,,,
1,GEORGE W. JOHNSON CENTER,1.0,"POLYGON ((11822517.112 6987684.108, 11822557.0...","(11822640.598755237, 6987512.569608039)"
2,DAVID KING HALL,1.0,"POLYGON ((11822800.554 6987724.175, 11822753.8...","(11822864.483255226, 6987750.865497605)"
3,SCIENCE AND TECH I,1.0,"POLYGON ((11823106.160 6987428.086, 11823006.5...","(11823017.541769873, 6987488.482499858)"
4,SCIENCE AND TECH II,1.0,"POLYGON ((11823144.777 6987485.867, 11823154.0...","(11823215.098050594, 6987415.048930175)"
5,ENTERPRISE HALL,1.0,"POLYGON ((11822946.564 6987144.449, 11822939.8...","(11823075.806553971, 6987192.066489626)"


# GMU Walkway

In [8]:
gmu_walkway = gpd.read_file("../data/raw/campus/Mason_walkway_line.shp").set_index("Id").set_crs("epsg:2283", allow_override=True)
gmu_walkway.head()

,geometry
Id,
0,"LINESTRING (11822464.147 6988476.896, 11822594..."
0,"LINESTRING (11822594.106 6988242.269, 11822624..."
0,"LINESTRING (11822329.189 6988193.984, 11822551..."
0,"LINESTRING (11822698.073 6987962.357, 11822695..."
0,"LINESTRING (11822695.074 6987952.560, 11822666..."


In [9]:
from shapely.geometry import Point
import uuid

vertex_set = set()
for _, row in gmu_walkway.iterrows():
    for point in row["geometry"].coords:
        vertex_set.add(point)

vertex_dict = {uuid.uuid4().int: Point(p) for p in vertex_set}
vertex_dict

{93991400257410126574193156630127279631: <shapely.geometry.point.Point at 0x10b27b6d0>,
 112860390218139369895066817649418648207: <shapely.geometry.point.Point at 0x135cf9ee0>,
 319255798756784672143135946032363262571: <shapely.geometry.point.Point at 0x135f1c370>,
 186462937979257312615915263092430337984: <shapely.geometry.point.Point at 0x135f1c640>,
 201438645527219040443261435111798628938: <shapely.geometry.point.Point at 0x135f1c580>,
 148037955346393201260157563177021699078: <shapely.geometry.point.Point at 0x135f1c670>,
 79174274387708609589103767797542588873: <shapely.geometry.point.Point at 0x135f1c3d0>,
 180023059443235056043599587607644217784: <shapely.geometry.point.Point at 0x135f1c5e0>,
 51357903169664645259034106432989180000: <shapely.geometry.point.Point at 0x135f1c1f0>,
 119162710400261447942462561106750022235: <shapely.geometry.point.Point at 0x135f1c160>,
 78503290454434403175083935696890449141: <shapely.geometry.point.Point at 0x135f1c0d0>,
 820584393847938764355881

In [10]:
vertex_df = gpd.GeoDataFrame.from_dict(vertex_dict, orient="index").rename(columns={0: "geometry"}).set_crs("epsg:2283")
vertex_df.head()

,geometry
93991400257410126574193156630127279631,POINT (11823573.236 6988812.749)
112860390218139369895066817649418648207,POINT (11822603.538 6987915.808)
319255798756784672143135946032363262571,POINT (11822975.586 6987109.325)
186462937979257312615915263092430337984,POINT (11822182.773 6986986.969)
201438645527219040443261435111798628938,POINT (11822451.046 6988641.197)


In [11]:
vertex_df.iloc[0]

geometry    POINT (11823573.236 6988812.749)
Name: 93991400257410126574193156630127279631, dtype: geometry

In [12]:
Point(1.0, 2.0).coords[0]

(1.0, 2.0)

# GMU Lakes and Rivers

In [13]:
gmu_lakes = gpd.read_file("../data/raw/campus/hydrop.shp").set_crs("epsg:2283", allow_override=True)
gmu_lakes.index.names = ["Id"]
gmu_lakes

,HYDRONAME,VISIBLE,HYDSYM,FFX_ID,Shape_Leng,Shape_Area,geometry
Id,,,,,,,
0,None,Y,300.0,6355.0,1311.435955,80644.439167,"POLYGON ((11821899.220 6987008.450, 11821907.3..."


In [14]:
gmu_rivers = gpd.read_file("../data/raw/campus/hydrol.shp").set_crs("epsg:2283", allow_override=True)
gmu_rivers.index.names = ["Id"]
gmu_rivers

,HYDRONAME,VISIBLE,HYDSYM,FFX_FID,Shape_Leng,geometry
Id,,,,,,
0,None,Y,236,3698.0,74.014969,"LINESTRING (11821782.040 6988841.090, 11821779..."
1,None,Y,236,5880.0,111.738289,"LINESTRING (11821762.585 6988769.823, 11821759..."
2,None,N,236,5881.0,45.328278,"LINESTRING (11821686.860 6988639.950, 11821690..."
3,None,Y,236,5883.0,122.868718,"LINESTRING (11823443.860 6988704.310, 11823447..."
4,None,N,236,5884.0,26.800686,"LINESTRING (11823461.880 6988559.010, 11823458..."
5,None,N,236,5886.0,32.028240,"LINESTRING (11821550.227 6988432.108, 11821562..."
6,None,Y,236,5889.0,204.436407,"LINESTRING (11823461.880 6988559.010, 11823474..."
7,None,Y,236,5890.0,297.024416,"LINESTRING (11821686.860 6988639.950, 11821681..."
8,None,Y,236,5891.0,65.041018,"LINESTRING (11821580.120 6988420.610, 11821583..."


# GMU Driveway

In [15]:
gmu_driveway = gpd.read_file("../data/raw/campus/Mason_Rds.shp").set_index("Id").rename(columns={"NAME": "name"})\
    .set_crs("epsg:2283", allow_override=True)
gmu_driveway.head()

,name,geometry
Id,,
0,Patriot Circle,"POLYGON ((11822370.708 6989071.583, 11822534.3..."
0,Sandy Creek Way,"POLYGON ((11823635.971 6987456.742, 11823727.7..."
0,Rivanna River Way,"POLYGON ((11823747.089 6988296.467, 11823764.2..."
0,Buffalo Creek Court,"POLYGON ((11823707.378 6988266.640, 11823707.3..."
0,Mason Pond Drive,"POLYGON ((11821538.678 6987432.458, 11821538.5..."
